In [5]:
import numpy as np
import pandas as pd

In [ ]:
# Define the start, end, and step values
start = 0.05
end = 3
step = 0.1

# Create the 1D arrays for each dimension
points = np.arange(start, end + step, step)

# Create a 4D grid
grid = np.array(np.meshgrid(points, points, points, points, indexing='ij')).T.reshape(-1, 4)

# Check the shape and the first few points
print(f"Grid shape: {grid.shape}")
print(grid)

Grid shape: (12960000, 4)
[[0.05 0.05 0.05 0.05]
 [0.1  0.05 0.05 0.05]
 [0.15 0.05 0.05 0.05]
 ...
 [2.9  3.   3.   3.  ]
 [2.95 3.   3.   3.  ]
 [3.   3.   3.   3.  ]]


In [37]:
a = [9.6, 11.7, 1.7, 3.8]
b = [4.1, 12.2, 1.2, 8.4]

# flyatk - walkatk , airdef - gnddef
c = [-1.4, -2.5]

In [39]:
v_vals = []

In [40]:
for v in grid:
# for i in range(len(w0)):
#     for j in range(len(w1)):
#         for k in range(len(w2)):
#             for l in range(len(w3)):
#                 v0,v1,v2,v3 = w0[i],w1[j],w2[k],w3[l]

    v0,v1,v2,v3 = v[:]
    d = (
        (a[0] * v0 - b[2] * v2) * v0,
        (a[1] * v1 - b[3] * v3) * v1,
        (a[2] * v2 - b[0] * v0) * v2,
        (a[3] * v3 - b[1] * v1) * v3
    )
    
    eps = (1, 1)
    
    e0, e1 = abs(c[0] - (d[0] - d[1])), abs(c[1] - (d[2] - d[3]))
    
    if (e0 < eps[0] and e1 < eps[1]):
        v_vals.append((v, e0, e1))
        print(f"{v0:.2f},{v1:.2f},{v2:.2f},{v3:.2f} > \t", end="")
        print(', '.join(tuple(map(lambda x: f"{x:.1f}", d))), '\t|', f"{d[0]-d[1]:.1f}", f"{d[2]-d[3]:.1f}")
    

2.50,2.30,0.30,0.05 > 	59.1, 60.9, -2.9, -1.4 	| -1.8 -1.5
2.55,2.35,0.30,0.05 > 	61.5, 63.6, -3.0, -1.4 	| -2.1 -1.6
2.75,2.50,0.30,0.05 > 	71.6, 72.1, -3.2, -1.5 	| -0.5 -1.7
2.80,2.55,0.30,0.05 > 	74.3, 75.0, -3.3, -1.5 	| -0.8 -1.7
2.85,2.60,0.30,0.05 > 	77.0, 78.0, -3.4, -1.6 	| -1.0 -1.8
2.90,2.65,0.30,0.05 > 	79.7, 81.1, -3.4, -1.6 	| -1.4 -1.8
2.95,2.70,0.30,0.05 > 	82.5, 84.2, -3.5, -1.6 	| -1.7 -1.8
3.00,2.75,0.30,0.05 > 	85.3, 87.3, -3.5, -1.7 	| -2.0 -1.9
1.95,1.80,0.35,0.05 > 	35.7, 37.2, -2.6, -1.1 	| -1.5 -1.5
2.00,1.85,0.35,0.05 > 	37.6, 39.3, -2.7, -1.1 	| -1.7 -1.5
2.05,1.90,0.35,0.05 > 	39.5, 41.4, -2.7, -1.1 	| -2.0 -1.6
2.10,1.95,0.35,0.05 > 	41.5, 43.7, -2.8, -1.2 	| -2.2 -1.6
2.20,2.00,0.35,0.05 > 	45.5, 46.0, -2.9, -1.2 	| -0.4 -1.7
2.25,2.05,0.35,0.05 > 	47.7, 48.3, -3.0, -1.2 	| -0.7 -1.8
2.30,2.10,0.35,0.05 > 	49.8, 50.7, -3.1, -1.3 	| -0.9 -1.8
2.35,2.15,0.35,0.05 > 	52.0, 53.2, -3.2, -1.3 	| -1.2 -1.9
2.40,2.20,0.35,0.05 > 	54.3, 55.7, -3.2, -1.3 	| -1.4 -1

In [61]:
sorted(v_vals, key=lambda x: x[1]**2 + x[2]**2)[:3]

[(array([1.3 , 1.15, 2.  , 0.1 ]),
  np.float64(0.0032500000000017515),
  np.float64(0.004999999999999893)),
 (array([3.  , 2.8 , 2.5 , 0.55]),
  np.float64(0.008000000000004004),
  np.float64(0.013500000000000512)),
 (array([1.65, 1.45, 2.7 , 0.2 ]),
  np.float64(0.026750000000001162),
  np.float64(0.013500000000000512))]

### Analysis for threat values

Pipe output using `tee out.txt`

Extract using sed
```
sed -nE 's/^FlyAtk: ([0-9\.]+).*/\1/p' out.txt > flyatk
sed -nE 's/.*\| WalkAtk: ([0-9\.]+).*/\1/p' out.txt > walkatk
```

Then use pandas

In [6]:
files = ['../flyatk', '../walkatk', '../airdef', '../gnddef']

percentiles = (50, 75)

for f in files:
    try:
        d = pd.read_csv(f)
        print(f"{f}: {np.percentile(d.values, percentiles[0]):.1f}, {np.percentile(d.values, percentiles[1]):.1f}")
    except:
        pass

../flyatk: 14.2, 18.5
../walkatk: 13.8, 17.8


In [7]:
e = pd.read_csv('../roles')

In [12]:
np.unique_counts(e.values)

UniqueCountsResult(values=array(['DEFENSE', 'NEUTRAL', 'OFFENSE'], dtype=object), counts=array([52, 32, 95]))

In [2]:
"a b  c".split(' ', maxsplit=2)

['a', 'b', ' c']

In [73]:
?str.split

Signature: str.split(self, /, sep=None, maxsplit=-1)
Docstring:
Return a list of the substrings in the string, using sep as the separator string.

  sep
    The separator used to split the string.

    When set to None (the default value), will split on any whitespace
    character (including \n \r \t \f and spaces) and will discard
    empty strings from the result.
  maxsplit
    Maximum number of splits.
    -1 (the default value) means no limit.

Splitting starts at the front of the string and works to the end.

Note, str.split() is mainly useful for data that has been intentionally
delimited.  With natural text that includes punctuation, consider using
the regular expression module.
Type:      method_descriptor